# Explainer to the analysis of extreme events

Extreme events are evaluated on the example of inland floods in the Krummhoern region.
Such floods ocure when precipitation inputs much water in a short time period in the region.
When the catchment water storage outside of the canals (e.g. soils) is allready saturated, the canals need to store and transport the precipitaion water.
Such conditions typically ocure during the storm season in the winter months, when evaporation is low and the storms bring lot precipitation.

**The following shall describe the single elements which are considered in the modelling of the maximum inland flood level in future states. The actual model run is included in 03_storage_model_forecast.ipynb**

## Changes in climate drivers

Due to climate change the average global sea levels will rise e.g. due to the meltdown of glaciers and the antarctic ice shield.
This rise is predicted for different RCP-scenarios within the 6th IPCC report

In [ ]:
# evaluate three RCP
for RCP in ['rcp26','rcp45','rcp85']:
print(RCP)

#at the end of the century (2090)
for ClimateTime in ['2065']: # the value '2065' is a dummy, which actually reads SLR-values for the year 2090

# using different SLR states (IPCC 6 - SLR assessment)
SLRs = [35,122,221]  # SLR in 2030 SSP126, outer range of low/medium confidence

Beside the change in sea level, also the intensity of extreme events may change with climatic changes.
This change is evaluated by analyzing the 95% percentile (upper end) of the precipitation forecasts from different climate predictions (The analysis can be found in 02_max_extremes.ipynb). 

In [ ]:
# assuming differing increases in the total precipitation of a single extreme event resulting from differing climate predictions (--> see 02_max_extremes.ipynb)
prec_increases = [0.9, 1, 1.1]

## Uncertainty in single event realization

Predictions of tidal sea levels on hourly time steps in combination with heavy rainfall events are beyond the available climate prediction data.
Therefore a storyline approach is included in this analysis, which evaluates how observed events would have developed under differing sea level and changed intensity of heavy precipiation events.

Six events are selected from the available records of water levels within the Krummhoern region. Characteristics of these events need to be added in 04_event_selection.ipynb

In [ ]:
# evaluating different observed events, for which increased water levels in canals are known
for event in ['2012','2017-03','2018-01','2017-12','2017-02','2011']:

The analyzed climate predictions come at daily time steps.
From the recorded events with increased water levels we can see, that the maximum water levels are driven by processes on subdaily base, e.g. by tidal water levels and shirt lasting high intenisve precipitaion events.

To acount for the variability in precipitation events we aggregated/and re-disaggregated te observed precipitation signal, based on a method by Hannes Müller-Thomy.

In [ ]:
# assuming differing subdaily precipitation patterns (temporal aggregation / Disaggregation of precipitation signal -> HMT)
for raindissagg in data.filter(regex='Prec').columns:

The drainage from the catchment can be limited either by the pumps or by the canal flow capacity.
The parameter uncertainty range of the canal flow capacity is represented by a minimum and maximum acceptable parameter set and the best fitting parameter set as calibrated in 01_canal_flow_dh-Q.ipynb.

In [ ]:
# evaluate canal flow uncertainty (--> see 01_canal_flow_dh-Q.ipynb)
canal_par = [(1.112 , 4156.), (0.9946, 2142.), (1.045 , 2820.)]

## Management options

From stakeholder communication it is possible to realize two larger water storage polders within the catchment (Freepsumer Meer, water level increase in Großes Meer).
This additional water storage would increase the water storage volume by approximately 1.5, without changing the canal hydraulics (canal flow parameters).

When an extreme event situation is predicted by short term weather forecast it seems possible to lower the target water level in the canals, to increase the available storage volume for the event.
From observation of catchment water levels it seems possible to lower the water level within the canals by approximately 5 cm within three days before the event.

The installation of a larger/more powerfull pump at Knock could change the pumping chart and increase drainage capacity under increased sea levels.

In [ ]:
# evaluate management options
for advance_pump in [0,50]: # lower target water level by 5 cm before incoming event
for Canal_area in [4,6]:    # increase available storage in canals, by increasing water area (polder option in "Freepsumer Meer" and "Großes Meer") -> note, that canal hydraulics are not changed
for pump in ['BAU', 'enlarge']: # use current pumping chart or slightly adapted pump

## Running the inland flood model

For each event the forcing data is selected and modified to:
 - increase tide water level by sea level rise
 - smoothen the precipitaion signal by an 12-hourly rolling mean to account for water redistribution processes within the catchment
 - increase the precipiation signal by a factor which represents changes in the intensity of extreme precipitation events
 
With this forcing data it is finally possible to run the inland flood model.

In [ ]:
hourly_recharge = data[raindissagg][t1:t2]
tide = data['wl_Knock_Outer'][t1:t2] + slr     # add SLR to tide water level [mm]
hourly_recharge = hourly_recharge.rolling("12h").mean() # smoothen rainfall signal to account for water redistribution within catchment
hourly_recharge *= prec_increase  # scale hourly recharge by change in climate forcing

### finally run the model
x['h_store'], q_pump, x['h_min'], x['flow_rec'], pump_cost, x['store'] = drain_cap.storage_model(forcing_data = x,
                                                                                                                                     canal_par = z,
                                                                                                                                     v_store = 0,
                                                                                                                                     h_store_target = -1350,
                                                                                                                                     canal_area = Canal_area,
                                                                                                                                     h_forecast_pump = advance_pump,
                                                                                                                                     h_grad_pump_max = maxdh)